# experiment 1

- EDA 2, 3 을 확인해보면 중복된 데이터가 있는 것을 확인해볼 수 있었다. 이를 같은 사람으로 보지 않고, 같은 피처를 가진 개별 데이터로 보고 이 같은 피처들이 가진 타겟값의 평균을 파겟값으로 하여 중복 데이터는 삭제하였다.
- 이외 과정은 [Simple Baseline LGBM - LB 0.72728 - 최정명](https://dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=vote) 과 같다.
- 성능 `0.8076415252` 로 저하되었다.


In [138]:
from urllib.request import urlretrieve

urlretrieve('https://drive.google.com/uc?export=download&id=1XLVFI_sK0smRVVuT8XU2s-M3lJT-68sN', './open.zip')

('./open.zip', <http.client.HTTPMessage at 0x7f4c12efc5d0>)

In [139]:
!unzip ./open.zip

Archive:  ./open.zip
replace open/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: open/train.csv          
replace open/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace open/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           


In [355]:
import numpy as np
import pandas as pd

## 데이터 확인
### [데이터 변수 설명](https://www.dacon.io/competitions/official/235713/talkboard/402821/)
- index
- gender: 성별
- car: 차량 소유 여부
- reality: 부동산 소유 여부
- child_num: 자녀 수
- income_total: 연간 소득
- income_type: 소득 분류
  - ['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
- edu_type: 교육 수준
  - ['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']
- family_type: 결혼 여부
  - ['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']
- house_type: 생활 방식
  - ['Municipal apartment', 'House / apartment', 'With parents', 'Co-op apartment', 'Rented apartment', 'Office apartment']
- DAYS_BIRTH: 출생일
  - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미
- DAYS_EMPLOYED: 업무 시작일
  - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미. 양수 값은 고용되지 않은 상태를 의미함
- FLAG_MOBIL: 핸드폰 소유 여부
- work_phone: 업무용 전화 소유 여부
- phone: 전화 소유 여부
- email: 이메일 소유 여부
- occyp_type: 직업 유형													
- family_size: 가족 규모
- begin_month: 신용카드 발급 월
  - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미
- credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도
  - 낮을 수록 높은 신용의 신용카드 사용자를 의미함



In [356]:
train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')

In [357]:
train.info()  # occyp_type -> null data 갖는다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [358]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          10000 non-null  int64  
 1   gender         10000 non-null  object 
 2   car            10000 non-null  object 
 3   reality        10000 non-null  object 
 4   child_num      10000 non-null  int64  
 5   income_total   10000 non-null  float64
 6   income_type    10000 non-null  object 
 7   edu_type       10000 non-null  object 
 8   family_type    10000 non-null  object 
 9   house_type     10000 non-null  object 
 10  DAYS_BIRTH     10000 non-null  int64  
 11  DAYS_EMPLOYED  10000 non-null  int64  
 12  FLAG_MOBIL     10000 non-null  int64  
 13  work_phone     10000 non-null  int64  
 14  phone          10000 non-null  int64  
 15  email          10000 non-null  int64  
 16  occyp_type     6848 non-null   object 
 17  family_size    10000 non-null  float64
 18  begin_m

In [359]:
train.head()

index gender car reality  ...   occyp_type  family_size begin_month credit
0      0      F   N       N  ...          NaN          2.0        -6.0    1.0
1      1      F   N       Y  ...     Laborers          3.0        -5.0    1.0
2      2      M   Y       Y  ...     Managers          2.0       -22.0    2.0
3      3      F   N       Y  ...  Sales staff          2.0       -37.0    0.0
4      4      F   Y       Y  ...     Managers          2.0       -26.0    2.0

[5 rows x 20 columns]

In [360]:
# Student 매우 작음...
train['income_type'].value_counts()

Working                 13645
Commercial associate     6202
Pensioner                4449
State servant            2154
Student                     7
Name: income_type, dtype: int64

In [361]:
len(train['index'].unique())

26457

## 공유 코드(EDA 2, 3)의 중복된 데이터 관련_1
> 그냥 다른 사람으로 생각하고 credit은 중복 데이터의 평균의 반올림으로 해볼까

## 중복 데이터 credit 평균으로 처리

In [362]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [363]:
train1 = train.copy()

In [364]:
train1.head()

index gender car reality  ...   occyp_type  family_size begin_month credit
0      0      F   N       N  ...          NaN          2.0        -6.0    1.0
1      1      F   N       Y  ...     Laborers          3.0        -5.0    1.0
2      2      M   Y       Y  ...     Managers          2.0       -22.0    2.0
3      3      F   N       Y  ...  Sales staff          2.0       -37.0    0.0
4      4      F   Y       Y  ...     Managers          2.0       -26.0    2.0

[5 rows x 20 columns]

In [365]:
train1 = train1.drop('index', axis=1)

In [366]:
# 타겟값까지 중복되는 행
np.sum(train1.duplicated(keep=False))

3155

In [367]:
# 타겟값을 제외하고 중복되는 행
np.sum(train1.drop(["credit"], axis = 1).duplicated(keep = False))

4497

In [368]:
dup_index = train1.drop(["credit"], axis = 1).duplicated(keep = False)
train1_dup = train1[dup_index]

In [369]:
train1_dup.shape

(4497, 19)

In [370]:
id_cols = train1_dup.columns.drop('credit')
id_cols

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type',
       'family_size', 'begin_month'],
      dtype='object')

In [371]:
train1_dup['id'] = train1_dup[id_cols].astype('str').apply(lambda x: '-'.join(x), axis=1)

In [372]:
train1_dup['id']

1        F-N-Y-1-247500.0-Commercial associate-Secondar...
2        M-Y-Y-0-450000.0-Working-Higher education-Marr...
19       F-N-Y-0-180000.0-Working-Secondary / secondary...
21       F-N-N-0-157500.0-Pensioner-Secondary / seconda...
24       F-N-N-0-202500.0-Pensioner-Secondary / seconda...
                               ...                        
26430    F-N-Y-0-112500.0-Working-Incomplete higher-Civ...
26431    F-N-Y-0-225000.0-Pensioner-Secondary / seconda...
26432    F-Y-Y-0-72000.0-Pensioner-Secondary / secondar...
26446    F-N-Y-0-135000.0-Working-Secondary / secondary...
26451    F-N-Y-0-202500.0-Working-Higher education-Marr...
Name: id, Length: 4497, dtype: object

In [373]:
train1_dup['id'].value_counts()

F-Y-N-0-315000.0-Working-Higher education-Married-Co-op apartment--13064--2286-1-1-0-0-Medicine staff-2.0--33.0                        5
F-N-Y-0-135000.0-Pensioner-Secondary / secondary special-Separated-House / apartment--20752-365243-1-0-0-0-nan-1.0--23.0               5
F-Y-Y-0-112500.0-Working-Secondary / secondary special-Civil marriage-House / apartment--13686--1256-1-0-0-0-Cooking staff-2.0--2.0    5
M-N-Y-0-270000.0-Working-Secondary / secondary special-Married-House / apartment--16896--248-1-0-0-0-Laborers-2.0--2.0                 4
F-Y-Y-0-135000.0-Pensioner-Secondary / secondary special-Widow-House / apartment--22991-365243-1-0-0-0-nan-1.0--53.0                   4
                                                                                                                                      ..
F-N-Y-0-202500.0-Working-Higher education-Married-House / apartment--19444--2832-1-0-0-0-nan-2.0--56.0                                 2
F-Y-Y-1-360000.0-Working-Higher education

In [374]:
np.sum(train1_dup['id'].value_counts() == 2)

1960

In [375]:
np.sum(train1_dup['id'].value_counts() == 3)

162

In [376]:
np.sum(train1_dup['id'].value_counts() == 4)

19

In [377]:
np.sum(train1_dup['id'].value_counts() == 5)

3

In [378]:
train1_dup.groupby('id')['credit'].mean()

id
F-N-N-0-103500.0-Pensioner-Secondary / secondary special-Married-House / apartment--24346-365243-1-0-0-0-nan-2.0--57.0                               2.0
F-N-N-0-108000.0-Commercial associate-Secondary / secondary special-Married-House / apartment--15803--9029-1-0-1-0-High skill tech staff-2.0--4.0    1.5
F-N-N-0-112500.0-Commercial associate-Higher education-Married-With parents--19721--1539-1-0-0-0-Accountants-2.0--1.0                                0.5
F-N-N-0-112500.0-Commercial associate-Incomplete higher-Married-House / apartment--10177--2339-1-0-1-0-nan-2.0--56.0                                 2.0
F-N-N-0-112500.0-Commercial associate-Secondary / secondary special-Married-House / apartment--17892--470-1-0-1-0-Sales staff-2.0--9.0               2.0
                                                                                                                                                    ... 
M-Y-Y-3-180000.0-Working-Secondary / secondary special-Married-House / apartmen

In [379]:
len(train1_dup['id'].unique())

2144

In [380]:
np.sum(train1_dup['id'].duplicated())

2353

In [381]:
np.sum(train1_dup['id'].duplicated(keep=False))

4497

In [382]:
train1_clear = train1_dup[train1_dup['id'].duplicated()==False]

In [383]:
train1_clear.shape

(2144, 20)

In [384]:
train1_clear.head()

gender car  ... credit                                                 id
1       F   N  ...    1.0  F-N-Y-1-247500.0-Commercial associate-Secondar...
2       M   Y  ...    2.0  M-Y-Y-0-450000.0-Working-Higher education-Marr...
19      F   N  ...    2.0  F-N-Y-0-180000.0-Working-Secondary / secondary...
21      F   N  ...    2.0  F-N-N-0-157500.0-Pensioner-Secondary / seconda...
24      F   N  ...    2.0  F-N-N-0-202500.0-Pensioner-Secondary / seconda...

[5 rows x 20 columns]

In [385]:
dup_row_credit_mean = train1_dup.groupby('id')['credit'].mean()

In [386]:
dup_row_credit_mean[train1_clear['id']]

id
F-N-Y-1-247500.0-Commercial associate-Secondary / secondary special-Civil marriage-House / apartment--11380--1540-1-0-0-1-Laborers-3.0--5.0    0.500000
M-Y-Y-0-450000.0-Working-Higher education-Married-House / apartment--19087--4434-1-0-1-0-Managers-2.0--22.0                                    1.500000
F-N-Y-0-180000.0-Working-Secondary / secondary special-Married-House / apartment--13727--6031-1-0-0-0-nan-2.0--7.0                             2.000000
F-N-N-0-157500.0-Pensioner-Secondary / secondary special-Married-House / apartment--21253-365243-1-0-1-0-nan-2.0--7.0                          1.666667
F-N-N-0-202500.0-Pensioner-Secondary / secondary special-Single / not married-House / apartment--22361-365243-1-0-1-0-nan-1.0--5.0             1.000000
                                                                                                                                                 ...   
M-N-N-1-247500.0-Working-Secondary / secondary special-Single / not married-House / a

In [387]:
dup_row_credit_mean[train1_clear['id']].values

array([0.5, 1.5, 2. , ..., 0.5, 1.5, 0. ])

In [388]:
train1_clear['credit'] = dup_row_credit_mean[train1_clear['id']].values

In [389]:
train1_clear.head()

gender car  ...    credit                                                 id
1       F   N  ...  0.500000  F-N-Y-1-247500.0-Commercial associate-Secondar...
2       M   Y  ...  1.500000  M-Y-Y-0-450000.0-Working-Higher education-Marr...
19      F   N  ...  2.000000  F-N-Y-0-180000.0-Working-Secondary / secondary...
21      F   N  ...  1.666667  F-N-N-0-157500.0-Pensioner-Secondary / seconda...
24      F   N  ...  1.000000  F-N-N-0-202500.0-Pensioner-Secondary / seconda...

[5 rows x 20 columns]

In [390]:
train1_clear = train1_clear.drop('id', axis=1)

In [391]:
type(round(1.23))

int

In [392]:
train1_clear['credit'] = train1_clear['credit'].apply(lambda x: round(x))

In [393]:
train1_clear['credit'] = train1_clear['credit'].astype('float64')

In [394]:
train1 = train1[dup_index == False]

In [395]:
train1.shape

(21960, 19)

In [396]:
train1 = pd.concat([train1, train1_clear])

In [397]:
train1.shape

(24104, 19)

In [398]:
train = train1.reset_index()

### test data preprocessing

In [409]:
test.shape

(10000, 19)

## 성능 비교
> [Simple Baseline LGBM - LB 0.72728 - 최정명](https://dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=vote) 와 똑같이 진행해서 더 나아지는지 확인

In [399]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

In [400]:
train.fillna('NAN', inplace=True)

In [414]:
test.fillna('NAN', inplace=True)

In [403]:
train.shape

(24104, 20)

In [402]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [404]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [415]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [406]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [407]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')


====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.665746	valid_1's multi_logloss: 0.768022
[200]	training's multi_logloss: 0.579349	valid_1's multi_logloss: 0.758956
Early stopping, best iteration is:
[268]	training's multi_logloss: 0.532086	valid_1's multi_logloss: 0.756957


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.666481	valid_1's multi_logloss: 0.77476
[200]	training's multi_logloss: 0.581114	valid_1's multi_logloss: 0.768864
Early stopping, best iteration is:
[215]	training's multi_logloss: 0.570568	valid_1's multi_logloss: 0.767819


====================================3============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.665391	vali

In [420]:
submit = pd.read_csv('./open/sample_submission.csv')

In [421]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [423]:
submit.to_csv('submission_2105052155.csv', index=False) #0.8076415252